# Na aula anterior...

Uso de `cross_val_predict`



```python
cross_val_predict(clf_model, X_exp, y_depend, cv=skf, method='predict_proba')
```

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.pipeline import Pipeline

In [14]:
df = pd.read_csv('dados/titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [46]:
var_expl = ['Pclass', 'Age', 'SibSp', 'Parch', 'Sex']
var_resp = 'Survived'

df.dropna(subset = var_expl, inplace = True)
df.Sex = df.Sex == 'male'

df_treino, df_teste = train_test_split(df, test_size = 0.2, random_state = 3)

x_treino = df_treino[var_expl].copy()
x_teste = df_teste[var_expl].copy()

y_treino = df_treino[var_resp].copy()
y_teste = df_teste[var_resp].copy()

In [ ]:
from sklearn.preprocessing import FunctionTransformer
FunctionTransformer(pd.get_dummies)

In [ ]:
df_treino, df_teste = train_test_split(df, test_size = 0.2, random_state = 3)

x_treino = df_treino[var_expl].copy()
x_teste = df_teste[var_expl].copy()

y_treino = df_treino[var_resp].copy()
y_teste = df_teste[var_resp].copy()

In [148]:
pipe_1 = Pipeline(steps = [
    ('modelo', DecisionTreeClassifier(max_depth = 4, random_state = 1)) 
])

In [48]:
pipe_2 = Pipeline(steps = [
    ('minmax', MinMaxScaler()),
    ('modelo', KNeighborsClassifier()) 
])

In [213]:
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict

In [206]:
%time

kf = KFold(n_splits = 5, shuffle = True, random_state = 1)

metricas = []

for i_treino, i_teste in kf.split(x_treino):
    
    x_treino_kf = x_treino.iloc[i_treino]
    y_treino_kf = y_treino.iloc[i_treino]
    
    x_val_kf = x_treino.iloc[i_teste]
    y_val_kf = y_treino.iloc[i_teste]
    
    pipe_1.fit(x_treino_kf, y_treino_kf)
    
    y_val_pred = pipe_1.predict_proba(x_val_kf)[:, 1]
    metricas.append(roc_auc_score(y_val_kf, y_val_pred))
    
np.mean(metricas)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.82 µs


0.7465486385714292

In [207]:
%time

kf = KFold(n_splits = 5, shuffle = True, random_state = 1)

metricas = []

for i_treino, i_teste in kf.split(x_treino):

    x_treino_kf = x_treino.iloc[i_treino]
    y_treino_kf = y_treino.iloc[i_treino]
    
    x_val_kf = x_treino.iloc[i_treino]
    y_val_kf = y_treino.iloc[i_treino]
    
    pipe_2 = Pipeline(steps = [
        ('minmax', MinMaxScaler()),
        ('modelo', KNeighborsClassifier()) 
    ])
    
    pipe_2.fit(x_treino_kf, y_treino_kf)
    
    y_val_pred = pipe_2.predict_proba(x_val_kf)[:, 1]
    metricas.append(roc_auc_score(y_val_kf, y_val_pred))
    
np.mean(metricas)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 9.06 µs


0.7296799533335021

In [198]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 1)

In [208]:
np.mean(cross_val_score(pipe_1, x_treino, y_treino, scoring = 'roc_auc', cv = kf))

0.7465486385714292

In [209]:
np.mean(cross_val_score(pipe_2, x_treino, y_treino, scoring = 'roc_auc', cv = kf))

0.7296799533335021

In [217]:
cross_val_predict(pipe_2, x_treino, y_treino, cv = kf, method = 'predict_proba')[:, 1]

array([0.2, 0. , 0. , 0.6, 0.4, 0. , 0.4, 0.8, 0.2, 0.6, 0.8, 0.8, 0.2,
       0.4, 0.4, 0.8, 0.6, 0. , 0. , 0.2, 0.2, 0.4, 0.8, 0. , 0.6, 0. ,
       0.2, 0.2, 0.2, 0. , 1. , 0. , 0.6, 0.4, 0.2, 0.6, 0. , 1. , 0.6,
       1. , 0.8, 1. , 0. , 0.4, 0.2, 0.2, 0.6, 0.4, 0.2, 0.8, 1. , 0.8,
       0.6, 1. , 0.6, 0.2, 0.4, 0.8, 0. , 0.4, 0.6, 0.4, 0.2, 0. , 0.4,
       0.6, 0. , 0.2, 0. , 0. , 0.2, 0.6, 0.4, 0.6, 0. , 0.6, 0.4, 0.8,
       0.2, 0. , 0.6, 0.6, 0.2, 0.2, 0.4, 0. , 0. , 0.2, 0.2, 0.6, 0.4,
       0.4, 0.2, 0.6, 0.2, 0.2, 0.8, 0.2, 0.6, 1. , 0.4, 0.8, 0. , 0. ,
       0.4, 0.2, 0.4, 0.6, 0.2, 0.2, 0. , 0.2, 0.2, 0. , 0.4, 0.4, 1. ,
       0.6, 0. , 1. , 0.6, 0.2, 0. , 1. , 0.2, 0.8, 0. , 0.2, 0.8, 0.2,
       0.2, 0.2, 0.6, 0. , 0.2, 0.2, 0. , 0.4, 0.4, 0.4, 0.8, 0.6, 0.2,
       0.6, 0.6, 0. , 0.8, 0.2, 0.4, 0.4, 0.4, 0.2, 0.6, 0. , 0.8, 0.6,
       1. , 0.4, 0.6, 0.6, 0.6, 0.6, 0.8, 1. , 0.2, 0.4, 0.8, 0.8, 0. ,
       0.2, 0.4, 0.8, 0. , 0. , 0. , 0. , 0.2, 0.2, 0.4, 0.6, 0.

In [50]:
pipe_2.fit(x_treino, y_treino)
y_teste_pred_2 = pipe_2.predict_proba(x_teste)[:, 1]
roc_auc_score(y_teste, y_teste_pred_2)

0.733454398708636

In [39]:
0.8173427991886411 - 0.8134888438133874

0.0038539553752536815

In [45]:
0.7568664169787765 - 0.6920516021639618

0.06481481481481477

In [51]:
0.7309322033898304 - 0.733454398708636

-0.002522195318805598

## Criando nossos próprios Transformers

In [228]:
df = pd.read_csv('dados/titanic.csv')

var_expl = ['Pclass', 'Age', 'SibSp', 'Parch', 'Sex']
var_resp = 'Survived'

df.dropna(subset = var_expl, inplace = True)

df_treino, df_teste = train_test_split(df, test_size = 0.2, random_state = 3)

x_treino = df_treino[var_expl].copy()
x_teste = df_teste[var_expl].copy()

y_treino = df_treino[var_resp].copy()
y_teste = df_teste[var_resp].copy()

In [231]:
from sklearn.preprocessing import FunctionTransformer

def sexo_masculino(tbl):
    tbl['Sex'] = tbl['Sex'] == 'male'
    return tbl

transformador = FunctionTransformer(sexo_masculino)

In [ ]:
df_treino, df_teste = train_test_split(df, test_size = 0.2, random_state = 3)

x_treino = df_treino[var_expl].copy()
x_teste = df_teste[var_expl].copy()

y_treino = df_treino[var_resp].copy()
y_teste = df_teste[var_resp].copy()

In [235]:
pipe_1 = Pipeline(steps = [
    ('trata_sexo', FunctionTransformer(sexo_masculino)),
    ('modelo', DecisionTreeClassifier(max_depth = 4, random_state = 1)) 
])

In [237]:
pipe_1.fit(x_treino, y_treino)
y_teste_pred_2 = pipe_1.predict_proba(x_teste)[:, 1]
roc_auc_score(y_teste, y_teste_pred_2)

0.775964696223317

# Hoje!

- Random Forest seleciona aleatoriamente os exemplos em cada amostra.
- E se essa seleção não for aleatória?

<img src="https://i.stack.imgur.com/mQ9Np.png"/>

## Exercício

Vamos aplicar o AdaBoost em cima do conjunto do projeto.

In [241]:
from sklearn.ensemble import AdaBoostClassifier

*Implementamos um algoritmo de boosting com um classificador naive Bayes usando um método similar àquele para uma árvore de decisão. Apesar de o algoritmo alcançar maior precisão que o classificador naive Bayes em alguns domínios, a melhoria na precisão sobre o classificador naive Bayes em um grande conjunto de domínios naturais é muito marginal. A razão deve ser que implicitamente boosting requer instabilidade do sistema de aprendizado que usou o boosting.* (Zheng, 1998)


*Melhoria efetiva ao se usar naive Bayes pode ser obtida ao utilizar diferentes subconjuntos de atributos para cada classificador intermediários* (Faceli et al., 2019)
